In [1]:
from CarGenerator import CarGenerator, Direction
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
kitti_path = '/scratch/local/hdd/hizkia/kitti'
date = '2011_09_29'
reference_rectangle = (64, 128)
min_original_rectangle = (32, 64)
mean = None
cov = None
optimize_direction = Direction.HORIZONTAL

In [3]:
generator = CarGenerator(kitti_path, 
                         date=date, 
                         reference_rectangle=reference_rectangle, 
                         min_original_rectangle=min_original_rectangle,
                         mean=mean, cov=cov, 
                         optimize_direction=optimize_direction).load_dataset()

In [4]:
def _help(i, x):
    print(i,end='\r')
    return x.normalized_depth.ravel()

In [5]:
def calculate_EM_score(normalized_samples, cov):
    inv_cov = np.linalg.inv(cov)
    return sum(np.apply_along_axis(lambda x: x.T @ inv_cov @ x, 0, normalized_samples))

In [6]:
D = np.array([_help(i,x) for i,x in enumerate(generator)]).T

In [7]:
mean = D.mean(axis=1)
A = D - mean.reshape((-1,1))
cov = np.cov(A)

In [8]:
calculate_EM_score(A, cov)

12998651.780641317

In [9]:
generator = CarGenerator(kitti_path, 
                         date=date, 
                         reference_rectangle=reference_rectangle, 
                         min_original_rectangle=min_original_rectangle,
                         mean=mean, cov=cov, 
                         optimize_direction=optimize_direction).load_dataset()

In [10]:
D = np.array([_help(i,x) for i,x in enumerate(generator)]).T

In [11]:
mean = D.mean(axis=1)
A = D - mean.reshape((-1,1))
cov = np.cov(A)

In [12]:
calculate_EM_score(A, cov)

33555870.732443094

In [ ]:
mean = D.mean(axis = 1)
A = D - mean.reshape((-1,1))
cov = np.cov(A)

np.savez("/scratch/local/hdd/hizkia/data.npz", A=A, mean=mean, cov=cov)

# a = np.load("/scratch/local/hdd/hizkia/data.npz")
# A = a['A']
# mean = a['mean']
# cov = a['cov']

In [ ]:
import pickle
for i, car in enumerate(generator):
    with open(f'/scratch/local/hdd/hizkia/cars/{i}.pkl','wb') as output:
        pickle.dump(car, output, pickle.HIGHEST_PROTOCOL)
    print(i, end='\r')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import pickle

In [ ]:
class CarDataset(Dataset):
    def __init__(self, base_path):
        self.base_path = base_path
        
    def __len__(self):
        return len(os.listdir(self.base_path))
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        with open(f'/scratch/local/hdd/hizkia/cars/{idx}.pkl','rb') as input:
            car = pickle.load(input)
            
        return car

In [ ]:
cd = CarDataset('/scratch/local/hdd/hizkia/cars/')

In [ ]:
plt.imshow(cd[2].image)